In [ ]:
# Creates dictionary of models as combinations of basis vectors to approximate the mixed signal.

In [4]:
# General import
import numpy as np
import matplotlib.pyplot as plt
import json
# Import the local functions, plots, and utilities
from functions import find_shiftX_exhaust, shift, scale_complex
from functions import gen_base, get_clusters, shift_x, is_incluster
from plots import plt_clust_Xy, plt_clust_Xyy

In [5]:
# Read the data flies and import functions
f_prefx = '../data/inphase_quadrature_'
with open(f_path + f_prefx + 'data.json') as f:
    iqdata = np.array(json.load(f))
with open(f_path + f_prefx + 'noise.json') as f:
    iqnoise = np.array(json.load(f))
# Convert to the complex row vectors
iqdata = iqdata[:, 0, :] + 1j * iqdata[:, 1, :]
iqnoise = iqnoise[:, 0, :] + 1j * iqnoise[:, 1, :]

In [7]:
dbasis = get_clusters() # The key is the centroid index, the value is the vector of item indices
# print(dbasis)
cls_sizes = [0, 0, 0, 10]  # Set sample size for i-th collision
dset = gen_base(iqdata, iqnoise, dbasis, cls_sizes)
print(list(dset[0].keys())) # List of the keys in each data sample
# print(dset[0])

['data', 'label', 'source', 'basis', 'coeff', 'shift']


In [8]:
# Set one data sample to reconstruct
answer_y = 7
y = dset[answer_y]['data']
answer_X = dset[answer_y]['basis']
answer_coeff = dset[answer_y]['coeff']
answer_shift = dset[answer_y]['shift']
print(answer_X)

data_scaled = scale_complex(iqdata[answer_X], answer_coeff)
plt_clust_Xyy(data_scaled, y, y, answer_X, answer_y, 0)


[777 131 384]


In [9]:
cnt = 0
MAX_SHIFT = 9
MAX_BASIS = 5
MAX_MODELS = 200

n_basis = len(dbasis)  # 64
idx_A = list(dbasis.keys()) # Later we check the reconstructed signal with one of the cluster's signals
AT = iqdata[idx_A]
A = AT.transpose()
del AT, data_scaled, cls_sizes, dset

mdl = {frozenset(): {'err': np.inf,    # key is the features
                     'fea': list([]),  # features in the right order
                     'sft': list([]),  # shift for each feature, same order
                     'par': np.empty([])  # set of parameters for each feature
                     }}
mdl_new = mdl.copy()  # Updated dictionary for form the next cycle


In [10]:
while cnt < MAX_BASIS:
    cnt += 1
    for idx in mdl.keys():
        # print('Append to the indices:', idx)
        for j in range(n_basis):  # Exhaustive search in the set of all features of A
            # Append index of a feature to the new dictionary
            idx_new = frozenset(set(idx) | {j})
            if idx_new in mdl: continue # Already in the dictionary, drop it
            # Compute the error, parameters and add them to the dictionary values
            x = A[:, j]
            # Append the new feature to the existing matrix
            features = mdl[idx]['fea'].copy()
            shifts = mdl[idx]['sft'].copy()
            X = A[:, features]
            X = shift(X, shifts)  # Arrange the shift
            err_min, best_b, best_shift = find_shiftX_exhaust(X, x, y, MAX_SHIFT)
            features.append(j)
            shifts.append(best_shift)
            # In the vector best_b the last feature corresponds to the last item
            mdl_new.update({idx_new: {'err': err_min, 'fea': features, 'sft': shifts, 'par': best_b}})

            if cnt == MAX_BASIS:
                if set(answer_X) == {idx_A[i] for i in features}:
                    print('hit to the answer', 'answer_X =', set(answer_X), 'idx_A =', {idx_A[i] for i in features})
                    X = A[:, features]
                    X = shift(X, shifts)
                    y1 = X @ best_b
                    plt_clust_Xyy(X.transpose(), y, y1, answer_X, answer_y, 0)

    mdl = dict(sorted(mdl_new.items(), key=lambda i: i[1]['err'])[:MAX_MODELS])


In [11]:
cnt = 0
for idx in mdl:
    cnt += 1
    X = A[:, list(idx)]
    err = mdl[idx]['err']
    par = mdl[idx]['par']
    sft = mdl[idx]['sft']
    X1 = shift(X, sft)
    y1 = X1 @ par
    plt_clust_Xyy(X1.transpose(), y, y1, list(idx), 'E {:.3f}'.format(err), cnt)
